# Overview
You can run through the cells in this notebook one at a time and run real-time predictions on images from your phone

In [ ]:
from ipywebrtc import CameraStream, ImageRecorder # for capturing pictures
from keras.models import load_model # for loading models
from keras.applications.mobilenet_v2 import preprocess_input
from PIL import Image
import numpy as np
import io
dx_name_dict = {
    'nv': 'melanocytic nevi',
    'mel': 'melanoma',
    'bcc': 'basal cell carcinoma',
    'akiec': 'Actinic keratoses and intraepithelial carcinoma',
    'vasc': 'vascular lesions',
    'bkl': 'benign keratosis-like',
    'df': 'dermatofibroma'
}
dx_name_id_dict = {id: name for id, name in enumerate(dx_name_dict.values())}
dx_name_vec = np.array([dx_name_id_dict[k] for k in sorted(dx_name_id_dict.keys())])

In [ ]:
my_model = load_model('skin_model.h5', compile=False)
my_model.summary()

In [ ]:
camera = CameraStream(constraints=
                      {'facing_mode': 'user',
                       'audio': False,
                       'video': { 'width': 224, 'height': 224 }
                       })
camera

In [ ]:
image_recorder = ImageRecorder(stream=camera)
image_recorder

In [ ]:
single_img = Image.open(io.BytesIO(image_recorder.image.value))
single_img

In [ ]:
single_img_arr = np.expand_dims(np.array(single_img.convert('RGB')), 0)
nn_input_arr = preprocess_input(single_img_arr)
print(nn_input_arr.shape)

In [ ]:
nn_output_arr = my_model.predict(nn_input_arr)

# Predictions from the Model

In [ ]:
nn_output_arr

## Show the class

In [ ]:
print(f'{dx_name_vec[np.argmax(nn_output_arr)]}, {np.max(nn_output_arr):2.1%}')